In [1]:
# load libraries
import pandas as pd
import os

In [2]:
# write paths here
patient_path = os.path.join("..","raw","RA_mtx_patient_info.tsv")
bridge_path = os.path.join("..","raw","Parent_Child_Bridge_info.csv")
olink_path = os.path.join("..","raw","Q-04911_Sung_NPX_2023-02-24.csv")
metabolomics_path = os.path.join("..","raw","metabolon_raw_norm_preprocessed.tsv")
metabolomics_bridge = os.path.join("..","raw","RA_MTX_CLP_GLOBAL_PROFILE_child_parentID_Aug1.xlsx")

# load patient data
patients = pd.read_csv(patient_path, sep="\t")
# load bridge file
bridge = pd.read_csv(bridge_path)
# load olink data
olink = pd.read_csv(olink_path, sep=";")
# load metabolomics
metabolomics = pd.read_csv(metabolomics_path, delimiter='\t').T
metabolomics_bridge = pd.read_excel(metabolomics_bridge)

In [3]:
# Data processing and merging
#patients.head()

In [4]:
#olink[olink["NPX"] == 0]

In [5]:
#olink.head()
#olinkId + UniProt + Assay represents 1 protein

In [6]:
# make one protein ID col combining UniProt, and Assay
olink["ProteinID"] = "UniProt" + olink["UniProt"].astype(str) + "#" + olink["Assay"].astype(str) + olink["OlinkID"].astype(str)

In [7]:
len(olink["ProteinID"].unique())

2944

In [8]:
mask = ~olink['SampleID'].str.contains("CONTROL")
olink = olink.loc[mask] # filters out the control prevalences that are just for Olink program

In [9]:
#olink.loc[olink["ProteinID"] == "UniProtA2VDF0#FUOMOID30626"] # check why this one had nan values

In [10]:
#olink[olink["NPX"] == 0] # check why some NPX are 0

In [11]:
# pivot the data to all be on the sample level instead of protein by protein
proteins = olink[['SampleID', 'ProteinID', 'NPX']].pivot(index='SampleID', columns='ProteinID', values='NPX').reset_index()
proteins = proteins.dropna(axis=1)
#proteins.head()

In [12]:
# Dictionary to hold new columns
new_cols_dict = {}

# function to linearize the log base 2 data
def log2_to_linear(x):
    return 2 ** x

# Convert Log2 NPX to linear scale and store in dictionary
for col in proteins.columns:
    if col.startswith('UniProt'):
        linear_col_name = 'linear_' + col
        new_cols_dict[linear_col_name] = proteins[col].apply(log2_to_linear)

# Create a new DataFrame from the dictionary
new_cols_df = pd.DataFrame(new_cols_dict)

# Concatenate the new columns with the original DataFrame
proteins = pd.concat([proteins, new_cols_df], axis=1)

# Define the order of the new columns
new_cols = proteins.columns

# Reorder columns if necessary
proteins = proteins[new_cols]

# Print the modified DataFrame to check
#proteins.head()

In [13]:
# subset patient data to Baseline and Plasma samples
patients = patients.loc[(patients["visit"]=="P1V1") & (patients["sample_type"]=="Plasma")]
# check that we got the original 60 patient visits
patients.shape[0] == 60

True

In [14]:
metabolomics = pd.read_csv(metabolomics_path, delimiter='\t').T
# Set the first row as the column names
metabolomics.columns = metabolomics.iloc[0]
metabolomics = metabolomics[1:]

# reset index to be a new col for merge purposes
metabolomics.reset_index(inplace=True)
# name index as child_id
metabolomics.rename(columns={'index': 'child_id'}, inplace=True)
#metabolomics.head()

In [15]:
#metabolomics_bridge.head()

In [16]:
# make the foreign key cols in both patients and olink using bridge dictionary
bridge_dict = pd.Series(bridge["Child Sample Id"].values, index=bridge["Parent Sample Id"]).to_dict() # parent --> child
# Convert the keys and values to integers, and remove any nan entries
bridge_dict = {str(int(k)): str(int(v)) for k, v in bridge_dict.items() if not (pd.isna(k) or pd.isna(v))}
# insert blank col
patients.insert(1, "child_id_prot", '')
# map to new values
patients["child_id_prot"] = patients["sample_id"].map(bridge_dict)
patients.rename(columns={"sample_id":"parent_id"}, inplace=True)

In [17]:
# repeat for olink
bridge_dict = pd.Series(bridge["Parent Sample Id"].values, index=bridge["Child Sample Id"]).to_dict() # child --> parent
# Convert the keys and values to integers, and remove any nan entries
bridge_dict = {str(int(k)): str(int(v)) for k, v in bridge_dict.items() if not (pd.isna(k) or pd.isna(v))}
# insert blank col
proteins.insert(1, "parent_id", '')
# map to new values
proteins["parent_id"] = proteins["SampleID"].map(bridge_dict)
proteins.rename(columns={"SampleID":"child_id_prot"}, inplace=True)

In [18]:
#proteins.head()

In [19]:
# make the foreign key cols in both patients and metabolomics using bridge dictionary
m_bridge_dict = pd.Series(metabolomics_bridge["Child Sample Id"].values, index=metabolomics_bridge["Parent Sample Id"]).to_dict() # parent --> child
# Convert the keys and values to integers, and remove any nan entries
m_bridge_dict = {str(int(k)): str(int(v)) for k, v in m_bridge_dict.items() if not (pd.isna(k) or pd.isna(v))}

# insert blank col
patients.insert(1, "child_id_met", '')
# map to new values
patients["child_id_met"] = patients["parent_id"].map(m_bridge_dict)
patients.rename(columns={"sample_id":"parent_id"}, inplace=True)

In [20]:
# add the metabolomics parent ID using the child ID
m_bridge_dict = pd.Series(metabolomics_bridge["Parent Sample Id"].values, index=metabolomics_bridge["Child Sample Id"]).to_dict() # child --> parent
# Convert the keys and values to integers, and remove any nan entries
m_bridge_dict = {str(int(k)): str(int(v)) for k, v in m_bridge_dict.items() if not (pd.isna(k) or pd.isna(v))}

# insert blank col
metabolomics.insert(1, "parent_id", '')
# map to new values
metabolomics["parent_id"] = metabolomics["child_id"].map(m_bridge_dict)
metabolomics.rename(columns={"child_id":"child_id_met"}, inplace=True)

In [21]:
# subset to just the demographic cols of interest before merging #
# create list of all to drop from merged metabolomics data
#demo_cols_all = ['child_id', 'parent_id', 'mc_number', 'visit', 'sample_type', 'delta_visit', 'age', 'sex', 'calc_bmi', 'smoke_status', 'patient_global_assessment', 'accp', 'rheum_factor', 'esr', 'crp', 'physician_global_assesment', 'sjc28', 'tjc28', 'mtx_dose', 'mtx_units', 'mtx_freq', 'bsln_pred', 'dose_pred', 'freq_pred', 'visit_date', 'cdai', 'das28crp', 'disease_activity', 'flare_comorb___1', 'flare_comorb___2', 'flare_comorb___3', 'flare_comorb___4', 'flare_comorb___5', 'flare_comorb___6', 'flare_comorb___7', 'flare_comorb___8', 'flare_comorb___9', 'flare_comorb___10', 'race___1', 'race___2', 'race___3', 'race___4', 'race___5', 'race___6', 'race___7', 'race___8', 'patient_haq', 'general_comments_f_u', 'general_comments_bsl', 'comments_off', 'no_change_v2', 'note_mtx_issue', 'note_stool_issue', 'note_dmard_issue', 'note_misc_issue', 'note_consent', 'note_jcf']

# the cols we want to include for demographic ML
demo_cols_want = ['EAC_ID', 'child_id_met', 'child_id_prot', 'parent_id', 'mtx_response', 'age', 'sex', 'calc_bmi', 'smoke_status', 'accp', 'rheum_factor', 'das28crp']

# subset the demographics
# demo is this subset of patients
# NOTE: if you want the full demographics, proceed using patients to merge instead of demo
demo = patients[demo_cols_want]

# change smoke_status to binary (1:never and 2:former --> 0:non-smoker, 3:smoker --> 1:smoker)
demo.loc[demo["smoke_status"] != 3.0, "smoke_status"] = 0
demo.loc[demo["smoke_status"] == 3.0, "smoke_status"] = 1
# change accp and rheum_factor to binary (ordination doesn't change still, currently coded as 1s and 2s)
demo.loc[demo["accp"] == 1.0, "accp"] = 0
demo.loc[demo["accp"] == 2.0, "accp"] = 1
demo.loc[demo["rheum_factor"] == 1.0, "rheum_factor"] = 0
demo.loc[demo["rheum_factor"] == 2.0, "rheum_factor"] = 1

#demo.head()

In [22]:
# check for missing data
demo.isnull().sum()

EAC_ID           0
child_id_met     0
child_id_prot    0
parent_id        0
mtx_response     0
age              0
sex              0
calc_bmi         1
smoke_status     0
accp             1
rheum_factor     1
das28crp         0
dtype: int64

In [23]:
# impute missing values by median #
calc_bmi_med = demo['calc_bmi'].median()
accp_med = demo['accp'].median()
rheum_factor_med = demo['rheum_factor'].median()

no_accp = demo.loc[demo['accp'].isnull(), 'EAC_ID']
no_calc_bmi = demo.loc[demo['calc_bmi'].isnull(), 'EAC_ID']
no_rheum_factor = demo.loc[demo['rheum_factor'].isnull(), 'EAC_ID']

demo.loc[demo['calc_bmi'].isnull(), 'calc_bmi'] = calc_bmi_med
demo.loc[demo['accp'].isnull(), 'accp'] = accp_med
demo.loc[demo['rheum_factor'].isnull(), 'rheum_factor'] = rheum_factor_med

# UNCOMMENT TO SEE WHICH ARE MISSING
#print('calc_bmi missing from idx ' + str(no_calc_bmi) + ' and imputed as ' + str(calc_bmi_med))
#print('accp missing from idx ' + str(no_accp) + ' and imputed as ' + str(accp_med))
#print('rheum_factor missing from idx ' + str(no_rheum_factor) + ' and imputed as ' + str(rheum_factor_med))

In [24]:
# subset to linear and log scale for proteomics
linear_protein_cols = [x for x in proteins.columns.to_list() if 'linear_UniProt' in x] # linearized data
log_protein_cols = [x for x in proteins.columns.to_list() if 'UniProt' in x and 'linear_UniProt' not in x] # original log2 data

# keep id cols for merging
also_keep_cols = ["child_id_prot", "parent_id"]

linear_protein_cols = also_keep_cols + linear_protein_cols
log_protein_cols = also_keep_cols + log_protein_cols

linear_proteins = proteins[linear_protein_cols]
log_proteins = proteins[log_protein_cols]

In [25]:
# create binary response col
demo.insert(4, "mtx_binary", 0)
demo.loc[demo["mtx_response"] != "none", "mtx_binary"] = 1
demo.drop("mtx_response", axis=1, inplace=True)

/var/folders/dl/5lw3wzn54kg220nzcdqxsvt00000gr/T/ipykernel_9867/2254982004.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo.drop("mtx_response", axis=1, inplace=True)


In [26]:
# merge into one dataset
# NOTE (same one again): if you want the full demographics, proceed using patients to merge instead of demo
# additionally, would need to deal with missing vals after if using the whole set
# like so (uncomment):
# data = pd.merge(patients, proteins, on="child_id_prot", how="left")
# data = pd.merge(data, metabolomics, on="child_id_met", how="left")

# demographics + (LINEAR AND LOG) proteomics + metabolomics
# This is just so everything is stored together
data = pd.merge(demo, proteins, on="child_id_prot", how="left")
data = pd.merge(data, metabolomics, on="child_id_met", how="left")

# demographics + LINEAR proteomics + metabolomics
linear_data = pd.merge(demo, linear_proteins, on="child_id_prot", how="left")
linear_data = pd.merge(linear_data, metabolomics, on="child_id_met", how="left")

# demographics + LOG proteomics + metabolomics
log_data = pd.merge(demo, log_proteins, on="child_id_prot", how="left")
log_data = pd.merge(log_data, metabolomics, on="child_id_met", how="left")

# retrieve the child_ids that merge to subset to just baseline, used as intermediate tables
child_id_prot = demo[["EAC_ID", "child_id_prot", "mtx_binary"]] # used mainly to subset and include the response column
child_id_met = demo[["EAC_ID", "child_id_met", "mtx_binary"]]

# proteins + metabolomics
lin_prot_met = pd.merge(linear_proteins, metabolomics, on="parent_id", how="left")
lin_prot_met = pd.merge(child_id_prot, lin_prot_met, on="child_id_prot", how = "left")

log_prot_met = pd.merge(log_proteins, metabolomics, on="parent_id", how="left")
log_prot_met = pd.merge(child_id_prot, log_prot_met, on="child_id_prot", how = "left")

# demographics + proteins
demo_lin_prot = pd.merge(demo, linear_proteins, on="child_id_prot", how="left")
demo_log_prot = pd.merge(demo, log_proteins, on="child_id_prot", how="left")

# demographics + metabolomics
demo_met = pd.merge(demo, metabolomics, on="child_id_met", how="left")

# JUST linear proteomics but adding mtx_binary
linear_proteins = pd.merge(child_id_prot, linear_proteins, on="child_id_prot", how = "left")
# JUST log proteomics but adding mtx_binary
log_proteins = pd.merge(child_id_prot, log_proteins, on="child_id_prot", how = "left")
# JUST metabolomics but adding mtx_binary
metabolomics = pd.merge(child_id_met, metabolomics, on="child_id_met", how = "left")

# only 60 baseline obs and no missing values --> success
# don't worry as much about number of cols because this changes with id cols and merging (check separately)

In [27]:
# clean the extra columns created in the merge
data.drop("parent_id_y", axis=1, inplace=True) # happens three times in here, make only one copy of parent_id
data.drop("parent_id_x", axis=1, inplace=True) # happens three times in here, make only one copy of parent_id
# reorder so that both ids are by the front
col_reorder_list = [x for x in data.columns.to_list() if x != "parent_id"]
col_reorder_list.insert(1, "parent_id")
data = data[col_reorder_list]

#data.head()

In [28]:
# SAVE OUT #
# Ensure the output directory exists to store the full datasets
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
output_dir = os.path.join(parent_dir, 'processed', 'full_data')
os.makedirs(output_dir, exist_ok=True)

In [29]:
## Fully merged data ##
# THIS INCLUDES:
#   - Demographics: age, sex, calc_bmi*, smoke_status*, accp*, rheum_factor*, das28crp
#   - BOTH: Linearized Proteomics AND Log2 Proteomics (cannot directly plug into ML, proteomics represented twice.)
#   - Metabolomics
data.to_csv(os.path.join(output_dir, "proteomics_metabolomics_demographics_merged.csv"), index=False)

In [30]:
# drop unnecessary columns merged on/existing in rest of data
# these ids exist in the merged data and can be remerged on EAC_ID (for validation purposes if needed)
# only keeping EAC_ID, mtx_binary and dropping other ID columns
# get rid of parent_id and child_id columns, does not get rid of EAC_ID because of caps

def drop_parent_child_ids(df):
    return df[[x for x in df.columns.to_list() if 'id' not in x]]

# demo + linear prot + metabolomics
linear_data = drop_parent_child_ids(linear_data)
# demo + log prot + metabolomics
log_data = drop_parent_child_ids(log_data)
# linear prot + met
lin_prot_met = drop_parent_child_ids(lin_prot_met)
# log prot + met
log_prot_met = drop_parent_child_ids(log_prot_met)
# demo + linear prot
demo_lin_prot = drop_parent_child_ids(demo_lin_prot)
# demo + log prot
demo_log_prot = drop_parent_child_ids(demo_log_prot)
# demo + met
demo_met = drop_parent_child_ids(demo_met)
# linear prot
linear_proteins = drop_parent_child_ids(linear_proteins)
# log prot
log_proteins = drop_parent_child_ids(log_proteins)
# metabolomics
metabolomics = drop_parent_child_ids(metabolomics)
# demographics
demo = drop_parent_child_ids(demo)

In [31]:
#linear_data.head()

In [32]:
#log_data.head()

In [33]:
#lin_prot_met.head()

In [34]:
#log_prot_met.head()

In [35]:
#demo_lin_prot.head()

In [36]:
#demo_log_prot.head()

In [37]:
#demo_met.head()

In [38]:
#linear_proteins.head()

In [39]:
#log_proteins.head()

In [40]:
#metabolomics.head()

In [41]:
# format: EACID + mtx_binary + data
# This allows for splitting directly from here for ML purposes because mtx_binary is already stored in each dataset
# ALL #
# demo + linear prot + metabolomics
linear_data.to_csv(os.path.join(output_dir, "lin_proteomics_demographics_metabolomics.csv"), index=False)
# demo + log prot + metabolomics
log_data.to_csv(os.path.join(output_dir, "log_proteomics_demographics_metabolomics.csv"), index=False)

# PAIRWISE #
# linear prot + met
lin_prot_met.to_csv(os.path.join(output_dir, "lin_proteomics_metabolomics.csv"), index=False)
# log prot + met
log_prot_met.to_csv(os.path.join(output_dir, "log_proteomics_metabolomics.csv"), index=False)
# demo + linear prot
demo_lin_prot.to_csv(os.path.join(output_dir,"lin_proteomics_demographics.csv"), index=False)
# demo + log prot
demo_log_prot.to_csv(os.path.join(output_dir,"log_proteomics_demographics.csv"), index=False)
# demo + met
demo_met.to_csv(os.path.join(output_dir,"demographics_metabolomics.csv"), index=False)

# SINGLE OMICS/DEMOGRAPHPICS #
# linear prot
linear_proteins.to_csv(os.path.join(output_dir, "linear_proteins.csv"), index=False)
# log prot
log_proteins.to_csv(os.path.join(output_dir, "log_proteins.csv"), index=False)
# metabolomics
metabolomics.to_csv(os.path.join(output_dir, "metabolomics.csv"), index=False)
# demographics
demo.to_csv(os.path.join(output_dir, "demographics.csv"), index=False)